In [12]:
import logging
import pandas as pd
import wandb
import joblib
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

WANDB_API_KEY=os.environ.get('WANDB_API_KEY')

In [3]:
# Login to Weights & Biases
!wandb login --relogin $WANDB_API_KEY

wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\pv\_netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [4]:
# name of the artifact related to test dataset
artifact_test_name = "attendance_face_recognition/test.csv:latest"
# name of the model artifact
artifact_model_name = "attendance_face_recognition/model_export:latest"

In [5]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging obj
logger = logging.getLogger()

In [6]:
# initiate the wandb project
run = wandb.init(project="attendance_face_recognition",job_type="test")

wandb: Currently logged in as: thanvinh164 (thanvinh164-vinh) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [7]:
logger.info("Downloading and reading test artifact")
artifact = run.use_artifact(artifact_test_name)
test_data_path = artifact.download()  # Lưu toàn bộ files về local
df_test = pd.read_csv(os.path.join(test_data_path, "test.csv"))

# Extract the target from the features
logger.info("Extracting target from dataframe")
x_test = df_test.copy()
y_test = x_test.pop("512")

29-04-2025 23:53:03 Downloading and reading test artifact
wandb:   1 of 1 files downloaded.  
29-04-2025 23:53:11 Extracting target from dataframe


In [11]:
# Download inference artifact
logger.info("Downloading and load the exported model")
model_export_path = run.use_artifact(artifact_model_name).download()
model = joblib.load(os.path.join(model_export_path, "model_export"))

30-04-2025 00:07:51 Downloading and load the exported model
wandb:   1 of 1 files downloaded.  


In [13]:
# predict
logger.info("Infering")
predict = model.predict(x_test)

# Evaluation Metrics
logger.info("Test Evaluation metrics")
acc = accuracy_score(y_test, predict)

logger.info("Test Accuracy: {}".format(acc))

run.summary["Acc"] = acc

30-04-2025 00:09:25 Infering
30-04-2025 00:09:25 Test Evaluation metrics
30-04-2025 00:09:25 Test Accuracy: 0.9918032786885246


In [14]:
print(classification_report(y_test,predict))

              precision    recall  f1-score   support

    11222333       1.00      1.00      1.00        25
    11225430       1.00      0.96      0.98        25
    11226754       0.96      1.00      0.98        25
    11226767       1.00      1.00      1.00        22
    11226938       1.00      1.00      1.00        25

    accuracy                           0.99       122
   macro avg       0.99      0.99      0.99       122
weighted avg       0.99      0.99      0.99       122



In [15]:
run.finish()

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Acc,0.9918
